# VGA-Automator: Floor Plan to DXF Converter

이 노트북은 건축 도면을 Space Syntax VGA 분석용 DXF 파일로 자동 변환하는 전체 과정을 보여줍니다.

This notebook demonstrates the complete pipeline for converting floor plan images to DXF files for VGA analysis.

## 설치 / Installation

In [ ]:
# Clone repository
!git clone https://github.com/Peperominusone/VGA-Automator.git
%cd VGA-Automator

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

## YOLOv8 모델 다운로드 / Download YOLOv8 Model

sanatladkat/floor-plan-object-detection의 best.pt 모델을 다운로드합니다.

Download the best.pt model from sanatladkat/floor-plan-object-detection.

**참고 / Note**: 실제 모델 다운로드 URL은 저장소에서 확인해주세요.

In [ ]:
# Download model (update URL as needed)
# Option 1: Direct download if available
!wget -q https://github.com/sanatladkat/floor-plan-object-detection/raw/main/best.pt -O best.pt

# Option 2: Clone the model repository
# !git clone https://github.com/sanatladkat/floor-plan-object-detection.git
# !cp floor-plan-object-detection/best.pt .

# Verify model file exists
import os
if os.path.exists('best.pt'):
    print("✓ Model downloaded successfully")
else:
    print("✗ Model not found. Please download manually from:")
    print("  https://github.com/sanatladkat/floor-plan-object-detection")

## 이미지 업로드 / Upload Floor Plan Image

변환할 도면 이미지를 업로드합니다.

Upload your floor plan image for conversion.

In [ ]:
from google.colab import files
import os

# Upload image
print("도면 이미지를 선택해주세요 / Select your floor plan image:")
uploaded = files.upload()

# Get uploaded filename
input_filename = list(uploaded.keys())[0]
print(f"\n✓ Uploaded: {input_filename}")

## 이미지 미리보기 / Preview Image

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Load and display image
image = cv2.imread(input_filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.title('Original Floor Plan')
plt.axis('off')
plt.show()

print(f"Image size: {image.shape[1]} x {image.shape[0]} pixels")

## Step 1: 이미지 전처리 / Image Preprocessing

노이즈 제거 및 이진화를 수행합니다.

Perform denoising and binarization.

In [ ]:
from src.preprocessor import Preprocessor

# Initialize preprocessor
preprocessor = Preprocessor()

# Preprocess image
print("Preprocessing image...")
processed_image = preprocessor.preprocess(
    input_filename,
    denoise_strength=10,
    threshold_method='adaptive'
)

# Display results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].imshow(image_rgb)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(processed_image, cmap='gray')
axes[1].set_title('Preprocessed (Binary)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✓ Preprocessing complete")

## Step 2: 객체 인식 / Object Detection

YOLOv8 모델로 벽체, 문, 창문 등을 감지합니다.

Detect walls, doors, windows using YOLOv8 model.

In [ ]:
from src.detector import FloorPlanDetector

# Initialize detector
print("Loading YOLOv8 model...")
detector = FloorPlanDetector(model_path='best.pt', confidence=0.5)

# Detect objects
print("Detecting objects...")
detections = detector.detect(image)

# Print detection summary
print(f"\n✓ Detected {len(detections)} objects:")
class_counts = {}
for det in detections:
    class_name = det['class_name']
    class_counts[class_name] = class_counts.get(class_name, 0) + 1

for class_name, count in sorted(class_counts.items()):
    print(f"  - {class_name}: {count}")

# Visualize detections
vis_image = detector.visualize_detections(image)
vis_rgb = cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 10))
plt.imshow(vis_rgb)
plt.title('Detected Objects')
plt.axis('off')
plt.show()

## Step 3: 윤곽선 추출 / Contour Extraction

감지된 벽체에서 직선을 추출합니다.

Extract lines from detected walls.

In [ ]:
from src.contour_extractor import ContourExtractor

# Extract wall lines
print("Extracting wall lines...")
wall_detections = detector.filter_detections_by_class(['Wall', 'Curtain Wall'])
lines = detector.extract_lines_from_walls(
    processed_image,
    wall_detections,
    threshold=50,
    min_line_length=30
)

print(f"✓ Extracted {len(lines)} wall lines")

# Visualize lines
if lines:
    extractor = ContourExtractor()
    vis_lines = extractor.visualize_lines(image, lines)
    vis_lines_rgb = cv2.cvtColor(vis_lines, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(vis_lines_rgb)
    plt.title('Extracted Wall Lines')
    plt.axis('off')
    plt.show()

## Step 4: DXF 내보내기 / Export to DXF

감지된 객체를 DXF 파일로 내보냅니다.

Export detected objects to DXF file.

In [ ]:
from src.dxf_exporter import DXFExporter

# Create DXF exporter
exporter = DXFExporter(
    image_height=image.shape[0],
    scale_factor=1.0  # Adjust as needed
)

# Export to DXF
output_filename = 'output.dxf'
print(f"Exporting to {output_filename}...")

exporter.export_from_detections(
    detections,
    image,
    output_filename,
    doors_as_openings=True,  # Doors are passable in VGA
    as_bbox=False  # Use detailed contours
)

print(f"\n✓ DXF file created: {output_filename}")
print(f"  File size: {os.path.getsize(output_filename) / 1024:.2f} KB")

## DXF 파일 다운로드 / Download DXF File

생성된 DXF 파일을 다운로드합니다.

Download the generated DXF file.

In [ ]:
from google.colab import files

# Download DXF file
files.download(output_filename)
print(f"\n✓ Downloaded: {output_filename}")
print("\nNext steps:")
print("1. Open the DXF file in AutoCAD or similar CAD software")
print("2. Import to depthmapX for VGA analysis")
print("3. Run Visibility Graph Analysis")

## 디버그 이미지 다운로드 (선택사항) / Download Debug Images (Optional)

중간 과정 이미지를 다운로드합니다.

Download intermediate processing images.

In [ ]:
# Save debug images
cv2.imwrite('preprocessed.png', processed_image)
cv2.imwrite('detections.png', vis_image)

if lines:
    cv2.imwrite('lines.png', vis_lines)

print("Debug images saved. Download them:")

# Download debug images
files.download('preprocessed.png')
files.download('detections.png')

if lines:
    files.download('lines.png')

print("\n✓ All debug images downloaded")

## 완료 / Complete!

변환이 완료되었습니다. DXF 파일을 VGA 분석 도구에서 사용할 수 있습니다.

Conversion complete! You can now use the DXF file in VGA analysis tools.

### 추천 VGA 분석 도구 / Recommended VGA Tools:

1. **depthmapX**: https://github.com/SpaceGroupUCL/depthmapX
2. **Space Syntax**: https://www.spacesyntax.net/

### 참고 자료 / References:

- VGA-Automator: https://github.com/Peperominusone/VGA-Automator
- YOLO Model: https://github.com/sanatladkat/floor-plan-object-detection